In [1]:
import numpy as np
import pandas as pd
# sklearn utilities
import tensorflow as tf

from sklearn.datasets import load_boston
from sklearn.preprocessing import robust_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
county_area = pd.read_csv('county_area.csv')
county_area = county_area.drop(["Areaname"], axis=1)
county_area = county_area.rename(columns={"STCOU": "fips"})

In [3]:
county_mask_use = pd.read_csv('mask_use.csv')
county_mask_use = county_mask_use.rename(columns={"COUNTYFP": "fips"})

In [4]:
county_cases = pd.read_csv('county_cases.csv')
county_cases = county_cases.drop(["county","state"], axis=1)

In [5]:
county_pop = pd.read_csv('county_pop.csv', encoding='latin-1')
county_pop = county_pop.rename(columns={"COUNTY": "fips"})

In [6]:
def new_normal(table):
    normal = robust_scale(table)
    return normal

In [7]:
df = county_pop.merge(county_mask_use, how='left', on="fips")
df = df.merge(county_area, how="left", on="fips")
df['POP_DEN'] = df['POPESTIMATE2019'] / df['AREA']
df['POP_DENxALWAYS'] = df['POP_DEN'] * df['ALWAYS']
df['POP_DENxFREQUENTLY'] =df['POP_DEN'] * df['FREQUENTLY']
df['POP_DENxSOMETIMES'] = df['POP_DEN'] * df['SOMETIMES']
df['POP_DENxRARELY'] = df['POP_DEN'] * df['RARELY']
df['POP_DENxNEVER'] = df['POP_DEN'] * df['NEVER']
df['POPxALWAYS'] = df['POPESTIMATE2019'] * df['ALWAYS']
df['POPxFREQUENTLY'] =df['POPESTIMATE2019'] * df['FREQUENTLY']
df['POPxSOMETIMES'] = df['POPESTIMATE2019'] * df['SOMETIMES']
df['POPxRARELY'] = df['POPESTIMATE2019'] * df['RARELY']
df['POPxNEVER'] = df['POPESTIMATE2019'] * df['NEVER']

In [8]:
df = df.merge(county_cases, how="left", on="fips")
df.insert(0, 'Ones', 1)
column_titles = ["POPESTIMATE2019","NEVER","RARELY","SOMETIMES","FREQUENTLY","ALWAYS","AREA","POP_DEN","POP_DENxALWAYS","POP_DENxFREQUENTLY","POP_DENxSOMETIMES","POP_DENxRARELY","POP_DENxNEVER",'POPxALWAYS','POPxFREQUENTLY','POPxSOMETIMES','POPxRARELY','POPxNEVER']
new_df = df.reindex(columns=column_titles)
normal_all = new_normal(new_df)
normal_df = pd.DataFrame(normal_all)
normal_df.columns = column_titles
normal_df['fips'] = df['fips']
normal_df = normal_df.merge(county_cases, how="left", on="fips")
column_titles = ["fips","cases","deaths","POPESTIMATE2019","NEVER","RARELY","SOMETIMES","FREQUENTLY","ALWAYS","AREA","POP_DEN","POP_DENxALWAYS","POP_DENxFREQUENTLY","POP_DENxSOMETIMES","POP_DENxRARELY","POP_DENxNEVER",'POPxALWAYS','POPxFREQUENTLY','POPxSOMETIMES','POPxRARELY','POPxNEVER']
normal_df = normal_df.reindex(columns=column_titles)
normal_df

,fips,cases,deaths,POPESTIMATE2019,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,AREA,...,POP_DENxALWAYS,POP_DENxFREQUENTLY,POP_DENxSOMETIMES,POP_DENxRARELY,POP_DENxNEVER,POPxALWAYS,POPxFREQUENTLY,POPxSOMETIMES,POPxRARELY,POPxNEVER
0,1001,2059,31,0.531522,-0.189873,0.013333,0.230769,1.096386,-0.241733,-0.087260,...,0.392066,0.986415,0.686119,0.574735,0.322373,0.388149,0.963127,0.700074,0.560736,0.323106
1,1003,6658,69,3.482726,0.189873,-0.186667,-0.230769,1.433735,-0.278221,2.556161,...,0.524723,1.416847,0.544150,0.524210,0.961035,2.663221,5.688549,2.873971,2.793491,4.353203
2,1005,1033,9,-0.018339,-0.012658,0.640000,0.051282,-0.036145,-0.027366,0.470378,...,-0.133261,-0.142277,-0.126075,0.052016,-0.139840,-0.009778,-0.015499,0.016638,0.277259,-0.015276
3,1007,840,14,-0.058754,-0.607595,-0.520000,-0.256410,0.891566,0.342075,-0.048220,...,0.001306,0.089251,-0.111614,-0.256064,-0.307315,0.011821,0.091853,-0.106102,-0.272649,-0.327508
4,1009,1932,25,0.566030,-0.189873,0.546667,0.820513,-0.120482,-0.173318,-0.001487,...,0.387452,0.466975,1.008201,1.061257,0.294052,0.442616,0.516004,1.141521,1.168028,0.349958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3123,56037,462,2,0.293013,-0.088608,2.960000,1.461538,-0.698795,-1.044470,18.284960,...,-0.367398,-0.396004,-0.335278,-0.260007,-0.378054,-0.034024,0.088161,1.040339,2.625569,0.225214
3124,56039,747,1,-0.039887,0.341772,1.120000,0.564103,0.518072,-0.716078,6.634632,...,-0.352040,-0.355401,-0.338757,-0.307004,-0.335629,-0.139745,0.055329,0.136269,0.449436,0.133611
3125,56041,431,3,-0.096983,0.379747,2.733333,0.487179,0.036145,-1.062714,2.668729,...,-0.339340,-0.322609,-0.285016,-0.037933,-0.272172,-0.222502,-0.081348,0.039673,0.928522,0.069685
3126,56043,144,7,-0.316007,1.721519,1.093333,-0.602564,0.975904,-0.957811,2.957099,...,-0.368973,-0.375142,-0.396628,-0.356235,-0.308237,-0.319737,-0.248058,-0.349510,-0.161863,-0.031260


In [9]:
def split(X, Y, size):
    return train_test_split(X, Y, test_size=size)

In [10]:
# Split Data
normal_svm_df = normal_df.drop(['fips','cases','deaths'], axis=1)
cols = df.shape[1]

X = np.array(normal_svm_df) # iloc slicing function 
y = np.array(df['cases'])

In [11]:
X = X.reshape(X.shape[0], X.shape[1], 1)
print(X.shape)

(3128, 18, 1)


In [12]:
xtrain, xtest, ytrain, ytest=train_test_split(X, y, test_size=0.2) 

In [53]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(18,1)))
model.add(tf.keras.layers.Dense(100, activation="selu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1, activation="selu"))
model.compile(loss="mse", optimizer="Adam")
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 18, 100)           200       
_________________________________________________________________
batch_normalization_30 (Batc (None, 18, 100)           400       
_________________________________________________________________
dense_45 (Dense)             (None, 18, 1)             101       
Total params: 701
Trainable params: 501
Non-trainable params: 200
_________________________________________________________________


In [54]:
# Evaluate the model on the test set
model.evaluate(xtest, ytest, verbose=2)

20/20 - 0s - loss: 90337080.0000


90337080.0

In [44]:
history = model.fit(
    xtrain, 
    ytrain, 
    epochs=200, 
    validation_split=0.25, 
    batch_size=50, 
    verbose=0
)

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

plot_metric(history, 'loss')